# Projeto 1 - Ciência dos Dados

Nome: Anna Beatriz Lima

Nome: Pedro Rubens

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

___
## Importando Bibliotecas


In [92]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [93]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\AnnaBeatriz\INSPER\2020.1\cdados\Projeto1-ttw


In [94]:
# função para limpar as bases 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    tira_mencoes = re.sub("@[A-Za-z0-9_]+","", text)
    tira_links = re.sub(r'http\S+', '', tira_mencoes)
    punctuation = '[!-.:?;@\/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', tira_links)
    return text_subbed

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [95]:
samsung_data = pd.read_excel('samsung1.xlsx')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a samsung que consiste em uma marca de celulares, relógios e outros produtos voltados a tecnologia. Classificamos como relevante todos aqueles tweets que exibem alguma crítica ou sugestão. Coisas que são relevantes para a marca em si e sobre os produtos que a mesma oferece e classificamos como irrelevantes todos os tweets que contém informações superficiais e que não agregam nada para a marca.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### limpeza das bases e classificação

In [96]:
# classificação
sum_negativo = samsung_data['relevancia'] == 2.0
negativo = samsung_data.loc[sum_negativo, :]

sum_positivo = samsung_data['relevancia'] == 1.0
positivo = samsung_data.loc[sum_positivo, :]

sum_neutro = samsung_data['relevancia'] == 0.0
neutro = samsung_data.loc[sum_neutro, :]

In [97]:
positivo_text = ''
for t in positivo['Treinamento']:
    t = str(t)
    positivo_text += t
    
samsung_pos = cleanup(positivo_text.lower())

In [98]:
negativo_text = ''
for t in negativo['Treinamento']:
    t = str(t)
    negativo_text += t
    
samsung_neg = cleanup(negativo_text.lower())

In [99]:
neutro_text = ''
for t in neutro['Treinamento']:
    t = str(t)
    neutro_text += t
    
samsung_neu = cleanup(neutro_text.lower())

In [100]:
# ensinando o classificador para positivo
serie_samsung_pos = pd.Series(samsung_pos.split())
tabela_samsung_pos = serie_samsung_pos.value_counts()
tabela_samsung_pos_relativa = serie_samsung_pos.value_counts(True)
tabela_samsung_pos_relativa.sum()
tabela_samsung_pos_relativa["samsung"]

0.059343434343434344

In [101]:
# ensinando o classificador para negativo
serie_samsung_neg = pd.Series(samsung_neg.split())
tabela_samsung_neg = serie_samsung_neg.value_counts()
tabela_samsung_neg_relativa = serie_samsung_neg.value_counts(True)
tabela_samsung_neg_relativa.sum()
tabela_samsung_neg_relativa["samsung"]

0.047619047619047616

In [102]:
# ensinando o classificador para neutro
serie_samsung_neu = pd.Series(samsung_neu.split())
tabela_samsung_neu = serie_samsung_neu.value_counts()
tabela_samsung_neu_relativa = serie_samsung_neu.value_counts(True)
tabela_samsung_neu_relativa.sum()
tabela_samsung_neu_relativa["samsung"]

0.044142614601018676

In [103]:
# probabilidade de conter samsung
samsung = samsung_pos + samsung_neg + samsung_neu

In [104]:
# construindo uma tabela samsung
serie_samsung = pd.Series(samsung.split())
tabela_samsung_relativa = serie_samsung.value_counts(True)
tabela_samsung_relativa["samsung"]

0.047082348663879295

In [105]:
#interseçao de condições
set_posi = set(tabela_samsung_pos_relativa.index)
set_nega = set(tabela_samsung_neg_relativa.index)
set_neut = set(tabela_samsung_neu_relativa.index)

In [106]:
#testando para um tweet

frase = 'samsung me mandou um convite bacana para a pré venda do galaxy s20, aceitei...mas e aí cadê o dinheiro kkkkk sem falar do absurdo de caro que deve ser aqui no brasil'
frase = cleanup(frase.lower())

tabela_frase = pd.Series(frase.split())
tabela_frase_a = tabela_frase.value_counts(True)

In [107]:
# Probabilidades
#pneg = tabela_samsung_relativa[serie_samsung_neg]
#probneg = len(tabela_samsung)/len(serie_samsung_neg)

tabela_samsung_absolut = serie_samsung.value_counts()
tabela_samsung_absolut.sum()

ppos = tabela_samsung_pos.sum()/tabela_samsung_absolut.sum()
pneg = tabela_samsung_neg.sum()/tabela_samsung_absolut.sum()
pneu = tabela_samsung_neu.sum()/tabela_samsung_absolut.sum()


print(tabela_frase.values)

pposPO=tabela_samsung_pos_relativa[tabela_frase.values]
print(pposPO.prod())

pposNG=tabela_samsung_neg_relativa[tabela_frase.values]
print(pposNG.prod())

pposNT=tabela_samsung_neu_relativa[tabela_frase.values]
print(pposNT.prod())


pposf =pposPO.prod()*ppos
print(pposf)
pnegf =pposNG.prod()*pneg
print(pnegf)
pneuf =pposNT.prod()*pneu
print(pneuf)

['samsung' 'me' 'mandou' 'um' 'convite' 'bacana' 'para' 'a' 'pré' 'venda'
 'do' 'galaxy' 's20' 'aceitei' 'mas' 'e' 'aí' 'cadê' 'o' 'dinheiro'
 'kkkkk' 'sem' 'falar' 'do' 'absurdo' 'de' 'caro' 'que' 'deve' 'ser'
 'aqui' 'no' 'brasil']
1.2916203455484921e-42
1.4305544817694005e-83
9.219763761821485e-66
1.859595189373579e-43
3.0582295411030993e-84
5.92304038070844e-66


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [108]:
# Abrindo a base teste
base_teste = pd.ExcelFile('samsung1.xlsx')
df1 = pd.read_excel(base_teste,'Teste')

In [109]:
# Percorrendo linha a linha e calculando probabilidades
i=0
for e in df1.Teste: 
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
    tweets = pd.Series(x.split())
    
    posit = 1
    for p in tweets:
        posit *= tabela_samsung_pos_relativa.get(p, 1)

    negat = 1    
    for p in tweets:
        negat *= tabela_samsung_neg_relativa.get(p, 1)
    
    neutr = 1
    for p in tweets:
        neutr *= tabela_samsung_neu_relativa.get(p, 1)
  
    
    posit = ppos*posit
    negat = pneg*negat
    neutr = pneu*neutr
    
    if posit > negat and posit > neutr:
        classif = 1
    elif negat > posit and negat > neutr:
        classif = 2
    elif neutr > posit and neutr > negat:
        classif = 0
        
  
    df1.loc[i, "classif"] = classif
    i+=1
    
print(posit, negat,neutr)

print (df1)

4.307703531150129e-26 4.610259100337782e-28 1.687693559375596e-29
                                                 Teste  relevancia  classif
0    tânia rocha um pouquinho de tudo - free fire n...           0      2.0
1    cupom notebook e computadores -informática ace...           0      2.0
2    @samsungbrasil verdade 🥺😥, já pedi um galaxy w...           1      2.0
3    rt @purplesidestore: 💘promoção - cases bts (ip...           0      1.0
4    @samsungbrasil eh sério samsung tô sem tv há 3...           0      2.0
5    @phantomrenegado depois que parar de funcionar...           2      1.0
6    @asusbr bd! comprei um aparelho dia 03/03, hj ...           2      1.0
7    a samsung perguntando se eu quero saber tudo s...           0      1.0
8    @andreiamelody1 amg n beije os vendedores da s...           0      0.0
9    @samsungbrasil @lucassi28687778 kkkk samsung t...           2      1.0
10   a samsung tá de sacanagem que vai lançar o gal...           0      1.0
11   rapaziada\n\ntô v

In [110]:
# Comparação entre Classificação Manual e o uso do classificador
comparacao = (pd.crosstab(df1['relevancia'], df1['classif']))
comparacao

classif,0.0,1.0,2.0
relevancia,,,
0,7,106,75
1,2,11,13
2,4,18,4


In [111]:
# Cálculo da porcentagem de acerto do Classificador
j = 0
total = 0
while j < 240:
    if df1.relevancia[j] == df1.classif[j]:
        total += 1
        
    j+=1 
total

porc_acerto = total/len(df1)
porc_acerto

0.09166666666666666

___
### Conclusão

Buscamos encontrar nesse trabalho a opinião publica sobre a marca de celulares Samsung, para isso pegamos uma serie de tweets relacionados a marca.
Com a conclusão desse projeto, temos como resultado uma porcentagem bem abaixo do que era esperando. Com uma porcentagem de acerto por volta de 10% vimos que a grande maioria dos erros está nos tweets que nos colocamos o grau de relevância como “Neutro”.
    Como uma justificativa disso, podemos observar que o classificador possui uma margem de erro muito grande quando comparado a novos métodos voltados a ciência dos dados.
    De modo a minimizar esse erro uma alternativa serie utilizar uma base maior de tweets para treinar nosso classificador.

___
### Outras aplicações

Outras aplicações do  método de classificação de Naive Bayes podem ser encontradas por exemplo na análise de dados de exames laboratoriais, e também em caixa de entrada de emails.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**